In [ ]:
# Trial of What is Happening

from keithley2600 import Keithley2600
from pmu_som import pmu_ctrl, pmu_cal, pmu_calc, pmu_def
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
from si_prefix import si_format

try:
    del(p.c)
except:
    pass

k = Keithley2600('GPIB0::17::INSTR', visa_library='')
k.smua.reset()
k.smuab.reset()

p = pmu_ctrl.pmu()

k.smua.measure.nplc = 3
k.smub.measure.nplce = 25

k.smua.source.output = k.smua.OUTPUT_OFF
k.smub.source.output = k.smub.OUTPUT_OFF
k.dispay.screen = k.display.USER
k.display.clear()
k.display.settext("Kandathil & Co")
k.display.settext("just testing the adc programming")


k.beeper.enable = k.beeper.ON

p.change_sys_ctrl({
    "DUTGND/CH" :1,
    "INT10K" :1
})






In [ ]:
ch = 0
p.write_all_PMU_REGS(0X1E060)

k.display.screen = k.display.SMUA
k.display.smua.measure.func = k.MEASURE_DCVOLTS

k.smua.measure.rangev       = 15
k.smua.measure.autorangei   = k.smua.AUTORANGE_ON
k.smua.measure.nplc         = 1
    
k.smua.source.func          = k.smua.OUTPUT_DCAMPS
k.smua.source.leveli        = 0
k.smua.source.limiti        = 10e-3
k.smua.source.limitv        = 14

p.change_sys_ctrl({
        "DUTGND/CH" : 1,
        "INT10K"    : 1,
        "GAIN"      : 2,
        "TMP ENABLE": 1,
        "TMP"       : 3 # Thermal shutdown at 100°C
    })
    
p.channels[ch].change_pmu_reg({
    "CH EN" : 1,     #Channel enable
    "FORCE" : 0,     #FV
    "C"     : 3,
    # "MEAS"  : 2,    #Temp Sensor (25°C = 1.5V | 4.6mV/°C)
    "MEAS"  : 1,    #VSens
    # "MEAS"  : 0,    #ISens
    "SS0"   : 1,
    "SF0"   : 1,
    "FIN"  : 1
})



In [2]:
#######################
#### caldata mem to pmu Testing by Jubin
#######################

# Creating Address List

#pmu_def.DAC_REG_TABLE ---->DAC_REG_TABLE = {    "Offset"    : 0,    (# only X1) ,
                                                # "FIN_I_5u"  : 8,  
                                                # "FIN_I_20u" : 9,  
                                                # "FIN_I_200u": 10,   
                                                # "FIN_I_2m"  : 11,   
                                                # "FIN_I_ext" : 12, 
                                                # "FIN_U"     : 13,   
                                                # "CLL_I"     : 20,  
                                                # "CLL_U"     : 21,  
                                                # "CLH_I"     : 28,  
                                                # "CLH_U"     : 29 }

#.keys retrns a view objects that display all the keys in the dictionary (DAC_REG_TABLE)----->    "Offset" "FIN_I_5u","FIN_I_20u","FIN_I_200u","FIN_I_2m" ,"FIN_I_ext","FIN_U","CLL_I", "CLL_U", "CLH_I" ,  "CLH_U" 
# [1:] ----> the first key will not be displayed ...... in this case "offset" will not be displayed
#  





adr_list = list()
for ch in range(0):
    for i,reg_name in enumerate(list(pmu_def.DAC_REG_TABLE.keys())[1:]):  
        print("Jubin") 


In [ ]:
# internal zero scale calibration for adc0
d = 1<<15 # decimal value is 32768


In [ ]:
## solo_adc_test trial

tmp_sens_cnf = 0X8232  #  writing the value 33,330 to the variable tmp_sens_cnf
print("ADC-ID  = 0x%04X" % p.c.command("adc_read -r 7 -l 16", read=True))


In [ ]:
# List of all commands that can be used for ADC (register address followed by the length)

#List of all ADC register with length

p.c.command("adc_read -r 7 -l 16") # Read ADC ID
p.c.command("adc_write -r 0X30 -l 24 -d 0X800000 ")  # Write the OFFSET0 with 0X800000
p.c.command("adc_read -r 0x30 -l 24", read=True) # Read OFFSET0
p.c.command("adc_read -r 0x38 -l 24", read=True) # Read GAIN0
                                                                                                                             


# Printing ADC ID
print("ADC-ID  = 0x%04X" % p.c.command("adc_read -r 7 -l 16", read=True))   # 0x%04X" this is for printing the output into a hexadecimal with 4 digits


DOING TRIAL

In [ ]:
tmp_sens_cnf = 0X8232
#Read the ADC ID
p.c.command("adc_read -r 7 -l 16 ", read =True)

# Read offset 0
offset_zero = p.c.command("adc_read -r 0X30  -l 24 ", read = True) - 0x800000 

# Read gain 0
gain_zero = p.c.command("adc_read -r 0X38  -l 24 ", read = True) / 0x400000
VREF = 2.5 # Mentioned in the datasheet




TRIAL - Internal zero scale calibration for adc0


In [ ]:
#do internal zero scale calibration for adc0
d =  1<<15
d |= 4<<4 # internal offset cal

p.c.command("adc_write -r 0x01 -l 16 -d %i" % d)

Trial - Test for single calibration

In [ ]:
# p.c.command("adc_write -r 0x30 -l 24 -d 0x800000") # writing the offset 0 with 0x800000

# Forcing 5 volt from PMU channel 0
p.channels[0].write_dac(pmu_calc.v)


In [ ]:
# Function for Voltage Conversion

def volt_conversion(val, Bipolar=True, gain=GAIN0, offset=OFFSET0):
    res = val#/gain

    if Bipolar:
        res -= 0x800000
    else:
        res /= 2
    
    # res += offset
    res *= VREF/0.75/(2**23)
    
    return res

In [ ]:
bipol =1

# This is the first step , This is setup for channel 0 ,like this we have to do for all channels
# Auch heisse channel confihuration

d =  1<<15 # Enable CH0
d |= 0<<12 # Enable SETUP for CHO
d |= 0x0<<5 # inp = ain0
d |= 0x4<<0 # inn = ain4 = 2.5V
p.c.command("adc_write -r 0x10 -l 16 -d %i" % d) # ch Register 0

d =  bipol<<12 # bipol is enabled and not unipol
d |= 1<<11 # refbuf+ enabled
d |= 1<<10 # refbuf- enabled
d |= 1<< 9 # ainbuf+ enabled
d |= 1<< 8 # ainbuf- enabled
d |= 2<<4  # internal 2.5 v reference is enabled
p.c.command("adc_write -r 0x20 -l 16 -d %i" %d) # SETUP CONFIGURATION REGISTER 0

p.c.command("adc_write -r 0x01 -l 16 -d 0x8010") #  ADC MODE REGISTER   - do single conversion

raw = p.c.command("adc_read -r 0x44 -l 24", True) # Get the converted data

# Converting the DAC vaue to voltage usin TOMS FUNCTION

conv = volt_conversion










In [ ]:
# Trying Internal Calibration
p.change_sys_ctrl({
     "DUTGND/CH" : 1,
     "INT10K"    : 1,
     "GAIN"      : 3, #1 #3
     "TMP ENABLE": 1,
     "TMP"       : 3 # Thermal shutdown at 100°C

})

k.smua.source.output = k.smua.OUTPUT_ON
k.display.settext("Ready to do $BStuff$N")
k.display.settext("Trying ADC Internal Calibration")

adc_meas = list()
smu_meas = list()
v_set = np.linspace(-11, 11, 25)
for v in tqdm(v_set):
    # k.smua.source.levelv = v
    p.channels[0].write_dac(pmu_calc.v_to_dac(v), "FIN_U")
    # sleep(0.1)
    p.c.command("adc_write -r 0x01 -l 16 -d 0x8010") # do single conversion
    # sleep(0.1)
    raw = p.c.command("adc_read -r 0x44 -l 24", True) # get data
    conv = volt_conversion(raw, bipol)
    conv *= 2/5     # seistive devider
    # conv *= 10      # gain
    adc_meas.append(conv)
    smu_meas.append(k.smua.measure.v())
    # print("A0_in = %sV" % si_format(conv, precision=3))
    
adc_meas = np.array(adc_meas)
smu_meas = np.array(smu_meas)

plt.plot(v_set,adc_meas, ".:", label="adc")
plt.plot(v_set,smu_meas, ".:", label="smu")
plt.legend()
plt.grid()
plt.show()


# Comparing Outputs for INTERNAL ZERO-SCALE CALIBRATION MODE,SYSTEM ZERO-SCALE CALIBRATION MODE AND SYSTEM FULL-SCALE CALIBRATION MODE

# Trying Internal Zero-Scale Calibration

d =  1<<15
d |= 4<<4 # internal offset cal

p.c.command("adc_write -r 0x01 -l 16 -d %i" % d)
OFFSET0 = p.c.command("adc_read -r 0x30 -l 24", read=True) - 0x800000
GAIN0 = p.c.command("adc_read -r 0x38 -l 24", read=True) / 0x400000

print(" \nChecking if the M and C Values are changed in the registers after Internal Zero Scale Calibration")
print("\n Offset after Internal zero Sccale calibration",OFFSET0)
print("\n gain after internal zero scale calibration  ", GAIN0)

adc_meas_after_cal = list()
smu_meas_after_cal = list()

# Doing Measurement to compare accuracy with SMU

for v in tqdm(v_set):
    p.channels[0].write_dac(pmu_calc.v_to_dac(V),"FIN_U")
    p.c.command("adc_write -r 0x01 -l 16 -d 0x8010")

    raw = p.c.command("adc_read -r 0x44 -l 24", True) # get data
    conv = volt_conversion(raw, bipol)
    conv *= 2/5     # seistive devider  - not exactly sure , why we are using this

    adc_meas_after_cal.append(conv)
    smu_meas_after_cal.append(k.smua.measure.v())


adc_meas_after_cal_internal_zero_scale_calibration = np.array(adc_meas_after_cal)
smu_meas_after_cal_internal_zero_scale_calibration = np.array(smu_meas_after_cal)

plt.plot(v_set,adc_meas_after_cal_internal_zero_scale_calibration,".:", label = "adc")
plt.plot(v_set,smu_meas_after_cal_internal_zero_scale_calibration,".:", label = "smu")
plt.legend()
plt.grid()
plt.show()
k.smua.source.output = k.smua.OUTPUT_OFF
















#Trying Offset Calibration of ADC using Interpolation


In [ ]:
# C calibration

p.change_sys_ctrl({

    "DUTGND/CH" : 1,
        "INT10K"    : 1,
        "GAIN"      : 3, #1 #3
        "TMP ENABLE": 1,
        "TMP"       : 3 # Thermal shutdown at 100°C

})

k.smua.source.output = k.smua.OUTPUT_ON
# adc_meas_low
# adc_meas_high


######################################################################################################################################################################################################################   
##################-------For default C Value (0x800000)-----------############################################################################################################################################################
######################################################################################################################################################################################################################

# Performing Measurement for x1 = 0X8000

c_val = p.c.command("adc_read -r 0x30 -l 24", read=True) - 0x800000
x1_val = 0x8000

ideal 




